In [ ]:
#!/usr/bin/env python3
"""
sales_share_cli_persistent.py

Features:
 - Base allocation: assign fixed base_amount_per_agent first (configurable).
 - Per-agent min_amount and max_amount thresholds (absolute currency amounts).
 - Iterative clamping + redistribution for variable portion to respect thresholds.
 - UI prompts for min/max when adding/editing agents; menu option to set base allocation.
 - Persistence of new fields in agents_data.json and history.
 - Delete agent option added.
"""
import json
from dataclasses import dataclass, asdict
from datetime import datetime
from typing import List, Dict, Optional
import os
import sys

# Try to use rich for pretty & colored tables. Fallback: tabulate.
USE_RICH = True
try:
    from rich.console import Console
    from rich.table import Table
    from rich import box
    console = Console()
except Exception:
    USE_RICH = False
    try:
        from tabulate import tabulate  # type: ignore
    except Exception:
        tabulate = None
        print("Warning: For pretty tables install 'rich' or 'tabulate' (pip install rich tabulate).")
        # We'll still proceed with simple prints.

DATA_FILE = "agents_data.json"
HISTORY_FILE = "distribution_history.json"

# Configurable cap for client points (5 points per client, cap at CLIENT_POINTS_CAP)
CLIENT_POINTS_PER_CLIENT = 5.0
CLIENT_POINTS_CAP = 100.0
DEFAULT_MAX_SENIORITY_MONTHS = 120  # 10 years cap by default

@dataclass
class Agent:
    id: int
    performance: float  # 0-100
    seniority_months: int
    target_points: float  # 0-100
    active_clients: int
    # new threshold fields (absolute amounts)
    min_amount: float = 0.0
    max_amount: Optional[float] = None

    def clients_points(self) -> float:
        pts = self.active_clients * CLIENT_POINTS_PER_CLIENT
        return min(pts, CLIENT_POINTS_CAP)

class SalesShareApp:
    def __init__(self):
        self.agents: List[Agent] = []
        self.weights = {
            "performance": 50.0,
            "target": 20.0,
            "seniority": 10.0,
            "clients": 20.0
        }
        self.max_seniority_months = DEFAULT_MAX_SENIORITY_MONTHS
        self.scale_seniority_using_max_agent = False
        self.base_amount_per_agent: float = 0.0  # new global base allocation per agent
        self.last_distribution: Optional[Dict[int, Dict]] = None
        self.load_data()

    # ---------------- Persistence ----------------
    def load_data(self):
        if not os.path.exists(DATA_FILE):
            # start fresh
            return
        try:
            with open(DATA_FILE, "r") as f:
                data = json.load(f)
            # load weights
            if "weights" in data:
                self.weights = data["weights"]
            if "max_seniority_months" in data:
                self.max_seniority_months = data["max_seniority_months"]
            if "scale_seniority_using_max_agent" in data:
                self.scale_seniority_using_max_agent = data["scale_seniority_using_max_agent"]
            if "base_amount_per_agent" in data:
                self.base_amount_per_agent = data["base_amount_per_agent"]
            # load agents (backward compatible: missing keys use dataclass defaults)
            self.agents = [Agent(**a) for a in data.get("agents", [])]
            print(f"Loaded {len(self.agents)} agent(s) from {DATA_FILE}.")
        except Exception as e:
            print(f"Failed to load data from {DATA_FILE}: {e}")

    def save_data(self):
        out = {
            "weights": self.weights,
            "max_seniority_months": self.max_seniority_months,
            "scale_seniority_using_max_agent": self.scale_seniority_using_max_agent,
            "base_amount_per_agent": self.base_amount_per_agent,
            "agents": [asdict(a) for a in self.agents]
        }
        try:
            with open(DATA_FILE, "w") as f:
                json.dump(out, f, indent=2)
        except Exception as e:
            print(f"Failed to save data to {DATA_FILE}: {e}")

    def append_history(self, distribution: Dict[int, Dict], total_amount: float):
        entry = {
            "timestamp": datetime.now().isoformat(),
            "total_amount": total_amount,
            "distribution": distribution
        }
        history = []
        if os.path.exists(HISTORY_FILE):
            try:
                with open(HISTORY_FILE, "r") as f:
                    history = json.load(f)
            except Exception:
                history = []
        history.append(entry)
        try:
            with open(HISTORY_FILE, "w") as f:
                json.dump(history, f, indent=2)
        except Exception as e:
            print(f"Failed to write history: {e}")

    # ---------------- Input helpers ----------------
    def input_float(self, prompt: str, minv=None, maxv=None, default=None):
        while True:
            s = input(prompt).strip()
            if s == "" and default is not None:
                return default
            try:
                v = float(s)
                if (minv is not None and v < minv) or (maxv is not None and v > maxv):
                    if maxv is None:
                        print(f"Enter a value >= {minv}.")
                    else:
                        print(f"Enter a value between {minv} and {maxv}.")
                    continue
                return v
            except ValueError:
                print("Enter a numeric value.")

    def input_int(self, prompt: str, minv=None, maxv=None, default=None):
        while True:
            s = input(prompt).strip()
            if s == "" and default is not None:
                return default
            try:
                v = int(s)
                if (minv is not None and v < minv) or (maxv is not None and v > maxv):
                    if maxv is None:
                        print(f"Enter an integer >= {minv}.")
                    else:
                        print(f"Enter an integer between {minv} and {maxv}.")
                    continue
                return v
            except ValueError:
                print("Enter an integer.")

    # ---------------- Agent management ----------------
    def add_agents(self, count: int):
        # compute start id as max existing id + 1 to avoid duplicates after deletes
        if self.agents:
            start_id = max(a.id for a in self.agents) + 1
        else:
            start_id = 1
        for i in range(count):
            aid = start_id + i
            print(f"\nEnter details for Agent ID {aid}:")
            perf = self.input_float("  Performance score (0-100): ", 0, 100)
            months = self.input_int("  Seniority (months): ", 0)
            targ = self.input_float("  Target achieved points (0-100): ", 0, 100)
            clients = self.input_int("  Number of active clients (integer): ", 0)
            # optional thresholds
            min_amt = self.input_float("  Minimum guaranteed amount for this agent [0]: ", 0, None, default=0.0)
            max_amt_input = input("  Maximum cap amount for this agent [none]: ").strip()
            max_amt = float(max_amt_input) if max_amt_input != "" else None
            self.agents.append(Agent(aid, perf, months, targ, clients, min_amount=min_amt, max_amount=max_amt))
        self.save_data()
        print(f"Added {count} agent(s).")

    def list_agents(self):
        if not self.agents:
            print("No agents yet.")
            return
        rows = []
        for a in self.agents:
            rows.append([
                a.id, a.performance, a.target_points, a.seniority_months,
                a.active_clients, a.clients_points(), a.min_amount, a.max_amount or ""
            ])
        headers = ["ID", "Perf", "Target", "Seniority(m)", "Clients", "ClientsPts", "MinAmt", "MaxAmt"]
        self.print_table(rows, headers)

    def edit_agent(self):
        if not self.agents:
            print("No agents to edit.")
            return
        aid = self.input_int("Enter agent ID to edit: ", 1)
        agent = next((a for a in self.agents if a.id == aid), None)
        if not agent:
            print("Agent not found.")
            return
        print("Press Enter to keep existing value.")
        perf = self.input_float(f" Performance ({agent.performance}): ", 0, 100, default=agent.performance)
        months = self.input_int(f" Seniority months ({agent.seniority_months}): ", 0, default=agent.seniority_months)
        targ = self.input_float(f" Target points ({agent.target_points}): ", 0, 100, default=agent.target_points)
        clients = self.input_int(f" Active clients ({agent.active_clients}): ", 0, default=agent.active_clients)
        min_amt = self.input_float(f" Min amount ({agent.min_amount}): ", 0, None, default=agent.min_amount)
        max_input = input(f" Max amount ({agent.max_amount if agent.max_amount is not None else 'none'}): ").strip()
        max_amt = float(max_input) if max_input != "" else agent.max_amount
        agent.performance, agent.seniority_months, agent.target_points, agent.active_clients = perf, months, targ, clients
        agent.min_amount, agent.max_amount = min_amt, max_amt
        self.save_data()
        print("Agent updated.")

    def delete_agent(self):
        if not self.agents:
            print("No agents to delete.")
            return
        self.list_agents()
        aid = self.input_int("Enter agent ID to delete: ", 1)
        agent = next((a for a in self.agents if a.id == aid), None)
        if not agent:
            print("Agent not found.")
            return
        confirm = input(f"Are you sure you want to delete Agent ID {aid}? (y/N): ").strip().lower()
        if confirm == "y":
            self.agents = [a for a in self.agents if a.id != aid]
            self.save_data()
            print(f"Agent ID {aid} deleted.")
        else:
            print("Deletion cancelled.")

    # ---------------- Weight / scaling management ----------------
    def normalized_weights(self) -> Dict[str, float]:
        s = sum(self.weights.values())
        if s == 0:
            return {k: 0.0 for k in self.weights}
        return {k: v / s for k, v in self.weights.items()}

    def set_weights(self):
        print("Current weights (will be normalized):")
        for k, v in self.weights.items():
            print(f"  {k}: {v}")
        print("Press Enter to keep existing value.")
        perf = self.input_float("New weight for performance: ", 0, None, default=self.weights["performance"])
        targ = self.input_float("New weight for target: ", 0, None, default=self.weights["target"])
        senior = self.input_float("New weight for seniority: ", 0, None, default=self.weights["seniority"])
        clients = self.input_float("New weight for clients: ", 0, None, default=self.weights["clients"])
        self.weights = {"performance": perf, "target": targ, "seniority": senior, "clients": clients}
        self.save_data()
        print("Weights updated.")

    def set_max_seniority(self):
        print(f"Current max_seniority_months: {self.max_seniority_months}")
        print(f"Scale using max months among agents? (currently {self.scale_seniority_using_max_agent})")
        ans = input("Use max among agents for scaling? (y/N): ").strip().lower()
        if ans == "y":
            self.scale_seniority_using_max_agent = True
            print("Will scale seniority using max months among agents.")
        else:
            self.scale_seniority_using_max_agent = False
            v = self.input_int("Enter max months to scale seniority (e.g., 120): ", 1, None, default=self.max_seniority_months)
            self.max_seniority_months = v
            print(f"Set max_seniority_months = {v}.")
        self.save_data()

    def set_base_allocation(self):
        print(f"Current base_amount_per_agent = {self.base_amount_per_agent}")
        v = self.input_float("Enter base amount per agent (absolute), 0 to disable: ", 0, None, default=self.base_amount_per_agent)
        self.base_amount_per_agent = v
        self.save_data()
        print(f"Set base_amount_per_agent = {v}")

    def edit_agent_thresholds(self):
        if not self.agents:
            print("No agents to edit.")
            return
        aid = self.input_int("Enter agent ID to set thresholds: ", 1)
        agent = next((a for a in self.agents if a.id == aid), None)
        if not agent:
            print("Agent not found.")
            return
        min_amt = self.input_float(f" Min amount ({agent.min_amount}): ", 0, None, default=agent.min_amount)
        max_input = input(f" Max amount ({agent.max_amount if agent.max_amount is not None else 'none'}): ").strip()
        max_amt = float(max_input) if max_input != "" else agent.max_amount
        agent.min_amount, agent.max_amount = min_amt, max_amt
        self.save_data()
        print("Agent thresholds updated.")

    # ---------------- Scoring & distribution ----------------
    def compute_agent_score(self, agent: Agent) -> float:
        w = self.normalized_weights()
        perf = agent.performance  # 0-100
        targ = agent.target_points  # 0-100
        clients_pts = agent.clients_points()  # clipped 0-100

        if self.scale_seniority_using_max_agent and self.agents:
            max_months = max(a.seniority_months for a in self.agents) or 1
        else:
            max_months = self.max_seniority_months or 1

        seniority_scaled = min((agent.seniority_months / max_months) * 100.0, 100.0)

        total = (
            perf * w["performance"]
            + targ * w["target"]
            + seniority_scaled * w["seniority"]
            + clients_pts * w["clients"]
        )
        return total

    def _iterative_clamp_redistribute(self, base_alloc_map: Dict[int, float], scores_map: Dict[int, float], total_remaining: float) -> Dict[int, float]:
        """
        Distribute `total_remaining` among agents proportionally to scores_map, but respecting
        absolute min/max totals (base + variable). base_alloc_map contains the already assigned base amounts (absolute).
        Returns a map: agent_id -> variable_amount (the part in addition to base).
        """
        # initialize
        agent_ids = list(scores_map.keys())
        var_alloc = {aid: 0.0 for aid in agent_ids}
        # compute target weights from scores (if all zero then use equal)
        remaining_ids = set(agent_ids)
        fixed_alloc = {}  # agents that hit min/max and are fixed
        # We'll work in terms of totals (base + var) when comparing to min/max
        # Iterative loop
        for _ in range(200):  # safety cap increased
            if not remaining_ids:
                break
            # compute sum of scores for remaining agents
            sum_scores = sum(scores_map[aid] for aid in remaining_ids)
            # provisional variable allocation for remaining agents
            provisional = {}
            if sum_scores <= 0:
                # split equally among remaining
                per = total_remaining / len(remaining_ids) if remaining_ids else 0.0
                for aid in remaining_ids:
                    provisional[aid] = per
            else:
                for aid in remaining_ids:
                    provisional[aid] = (scores_map[aid] / sum_scores) * total_remaining
            # check clamps
            clamp_happened = False
            for aid in list(remaining_ids):
                base = base_alloc_map.get(aid, 0.0)
                proposed_total = base + provisional[aid]
                agent = next((a for a in self.agents if a.id == aid), None)
                if agent is None:
                    # skip if agent missing
                    remaining_ids.remove(aid)
                    clamp_happened = True
                    continue
                # enforce min
                if proposed_total < agent.min_amount:
                    # fix at min, reduce total_remaining and remove from remaining_ids
                    var_part = max(agent.min_amount - base, 0.0)
                    fixed_alloc[aid] = var_part
                    total_remaining -= var_part
                    remaining_ids.remove(aid)
                    clamp_happened = True
                elif agent.max_amount is not None and proposed_total > agent.max_amount:
                    var_part = max(agent.max_amount - base, 0.0)
                    fixed_alloc[aid] = var_part
                    total_remaining -= var_part
                    remaining_ids.remove(aid)
                    clamp_happened = True
            if not clamp_happened:
                # no clamping this iteration — accept provisional for remaining
                for aid in remaining_ids:
                    var_alloc[aid] = provisional[aid]
                # combine with fixed_alloc
                for k, v in fixed_alloc.items():
                    var_alloc[k] = v
                return var_alloc
            # if clamps happened, loop will recompute provisional with updated remaining_ids & total_remaining
        # if we reach here, try to assign what's left
        if remaining_ids:
            per = total_remaining / len(remaining_ids)
            for aid in remaining_ids:
                var_alloc[aid] = per
        for k, v in fixed_alloc.items():
            var_alloc[k] = v
        return var_alloc

    def distribute_amount(self, total_amount: float) -> Dict[int, Dict]:
        """
        New flow:
         - Assign base_amount_per_agent to each agent (or scaled down equally if base * N > total_amount).
         - Remaining amount distributed by scores, with iterative clamping to respect per-agent min/max (applied to total).
        """
        n = len(self.agents)
        # handle degenerate
        if n == 0:
            return {}

        # Compute base allocation
        requested_base_total = self.base_amount_per_agent * n
        base_alloc_map = {}
        variable_alloc_map = {}
        if requested_base_total > total_amount and requested_base_total > 0:
            # scale base down so sum(base) == total_amount (no variable portion)
            effective_base_per_agent = total_amount / n
            for a in self.agents:
                base_alloc_map[a.id] = round(effective_base_per_agent, 2)
            # no remaining amount to distribute
            variable_alloc_map = {a.id: 0.0 for a in self.agents}
        else:
            for a in self.agents:
                base_alloc_map[a.id] = self.base_amount_per_agent
            remaining = total_amount - sum(base_alloc_map.values())
            # Compute scores
            scores = {a.id: self.compute_agent_score(a) for a in self.agents}
            # If total scores <= 0, remaining is split equally but thresholds still apply via iterative clamp
            variable_alloc_map = self._iterative_clamp_redistribute(base_alloc_map, scores, remaining)

        # Build final result and compute a 'score' metric for reporting (use compute_agent_score)
        result = {}
        for a in self.agents:
            base = base_alloc_map.get(a.id, 0.0)
            var = variable_alloc_map.get(a.id, 0.0)
            total_share = round(base + var, 2)
            # ensure max/min absolute safety (rounding can slightly violate)
            if a.max_amount is not None and total_share > a.max_amount:
                total_share = round(a.max_amount, 2)
            if total_share < a.min_amount:
                total_share = round(a.min_amount, 2)
            sc = round(self.compute_agent_score(a), 3)
            justification = (
                f"Base={base:.2f}, Var={var:.2f}, Perf={a.performance:.1f}, "
                f"Target={a.target_points:.1f}, SeniorityMonths={a.seniority_months}, "
                f"Clients={a.active_clients} -> weighted points={sc:.3f}"
            )
            result[a.id] = {"share": total_share, "score": sc, "justification": justification, "base": round(base,2), "variable": round(var,2)}
        return result

    # ---------------- Table printing ----------------
    def print_table(self, rows: List[List], headers: List[str], highlight_top_low: bool = False, score_map: Dict[int, float] = None):
        """
        If rich is available, use it with color coding. Otherwise, use tabulate or simple prints.
        highlight_top_low: if True, expects score_map to exist and will color the row with highest score green and lowest red.
        """
        if USE_RICH:
            table = Table(box=box.SIMPLE_HEAVY)
            for h in headers:
                table.add_column(h, justify="center")
            # determine top/low ids if requested
            top_id = low_id = None
            if highlight_top_low and score_map:
                # find id with max and min score
                sorted_scores = sorted(score_map.items(), key=lambda x: x[1])
                if sorted_scores:
                    low_id = sorted_scores[0][0]
                    top_id = sorted_scores[-1][0]
            for r in rows:
                # assume first col is ID
                aid = r[0]
                # stringify row
                row_strs = [str(x) for x in r]
                if highlight_top_low and score_map:
                    if aid == top_id:
                        table.add_row(*row_strs, style="bold green")
                    elif aid == low_id:
                        table.add_row(*row_strs, style="bold red")
                    else:
                        table.add_row(*row_strs)
                else:
                    table.add_row(*row_strs)
            console.print(table)
        else:
            # fallback to tabulate if available
            try:
                if tabulate:
                    print(tabulate(rows, headers=headers, tablefmt="grid"))
                else:
                    raise Exception("tabulate not available")
            except Exception:
                # final fallback - naive printing
                print(" | ".join(headers))
                for r in rows:
                    print(" | ".join(str(x) for x in r))

    # ---------------- Export ----------------
    def export_last_distribution(self, filename: str = "distribution_export.json"):
        if not self.last_distribution:
            print("No distribution computed yet.")
            return
        out = {
            "timestamp": datetime.now().isoformat(),
            "weights": self.weights,
            "max_seniority_months": self.max_seniority_months,
            "scale_seniority_using_max_agent": self.scale_seniority_using_max_agent,
            "base_amount_per_agent": self.base_amount_per_agent,
            "agents": [asdict(a) for a in self.agents],
            "distribution": self.last_distribution
        }
        try:
            with open(filename, "w") as f:
                json.dump(out, f, indent=2)
            print(f"Exported last distribution to {filename}")
        except Exception as e:
            print(f"Failed to export: {e}")

    # ---------------- Main actions ----------------
    def action_distribute(self):
        if not self.agents:
            print("No agents - add some first.")
            return
        amt = self.input_float("Total amount to distribute: ", 0)
        dist = self.distribute_amount(amt)
        self.last_distribution = dist
        # prepare rows for display. include score to identify top/low
        rows = []
        score_map = {}
        for a in self.agents:
            info = dist[a.id]
            rows.append([a.id, f"{info['share']:,}", info["score"], info.get("base", 0.0), info.get("variable",0.0),
                         a.performance, a.target_points, a.seniority_months, a.active_clients])
            score_map[a.id] = info["score"]
        headers = ["Agent ID", "Share", "Points", "Base", "Var", "Perf", "Target", "Seniority(m)", "Clients"]
        # print table highlighting top and low
        self.print_table(rows, headers, highlight_top_low=True, score_map=score_map)
        # print justifications
        print("\nJustifications:")
        for aid in sorted(dist.keys()):
            print(f" Agent {aid}: {dist[aid]['justification']}")
        total_given = sum(dist[aid]["share"] for aid in dist)
        print(f"\nTotal distributed: {round(total_given,2)} (requested {amt})")
        # persist
        self.append_history(dist, amt)
        # autosave last state (agents, weights)
        self.save_data()

    # ---------------- CLI loop ----------------
    def run(self):
        menu = """
Menu:
  1) Add agents
  2) List agents
  3) Edit agent
  4) Set/Change weights
  5) Set seniority scaling
  6) Distribute amount
  7) Export last distribution to JSON
  8) Show distribution history (last 10)
  9) Set global base allocation per agent
 10) Edit agent min/max thresholds
 11) Delete agent
 12) Exit
"""
        while True:
            print(menu)
            choice = input("Choose option: ").strip()
            if choice == "1":
                c = self.input_int("How many agents to create? ", 1)
                self.add_agents(c)
            elif choice == "2":
                self.list_agents()
            elif choice == "3":
                self.edit_agent()
            elif choice == "4":
                self.set_weights()
            elif choice == "5":
                self.set_max_seniority()
            elif choice == "6":
                self.action_distribute()
            elif choice == "7":
                fn = input("Filename [distribution_export.json]: ").strip() or "distribution_export.json"
                self.export_last_distribution(fn)
            elif choice == "8":
                self.show_history()
            elif choice == "9":
                self.set_base_allocation()
            elif choice == "10":
                self.edit_agent_thresholds()
            elif choice == "11":
                self.delete_agent()
            elif choice == "12":
                self.save_data()
                print("Exiting.")
                break
            else:
                print("Invalid option. Pick 1-12.")

    def show_history(self, limit: int = 10):
        if not os.path.exists(HISTORY_FILE):
            print("No distribution history found.")
            return
        try:
            with open(HISTORY_FILE, "r") as f:
                history = json.load(f)
        except Exception as e:
            print(f"Failed to load history: {e}")
            return
        if not history:
            print("History is empty.")
            return
        # show last `limit` entries
        tail = history[-limit:]
        rows = []
        for entry in reversed(tail):  # latest first
            ts = entry.get("timestamp", "")
            total_amount = entry.get("total_amount", "")
            dist = entry.get("distribution", {}) or {}
            if dist:
                try:
                    top_aid = max(dist.items(), key=lambda x: float(x[1].get("share", 0)))[0]
                    top_share = dist[top_aid].get("share", "")
                except Exception:
                    top_aid, top_share = "", ""
            else:
                top_aid, top_share = "", ""
            rows.append([ts, total_amount, top_aid, top_share])
        headers = ["Timestamp", "TotalAmount", "TopAgentID", "TopShare"]
        self.print_table(rows, headers)

# ---------------- Run App ----------------
if __name__ == "__main__":
    app = SalesShareApp()
    app.run()
